In [1]:
import os
import numpy as np
import scipy.io as sio

# Set type
reject_trials = True  # True is reject minus trial else keep it original

# Set epoch period
Ts = 1000  # Sampling rate
epoch_setting = np.array([-300, 300]) / 1000  # in seconds
eset = np.abs(np.round(epoch_setting * Ts)).astype(int)

# Load data list
path = r'C:\Users\Jisub\Desktop\Saccadic\left\data'
file_list = sorted(os.listdir(path))
del file_list[5]  # Remove 6th file

In [2]:
total_epoch = []

for fname in file_list:
    data = sio.loadmat(os.path.join(path, fname), squeeze_me=True)
    
    Saccade = data['Saccade']
    T0 = data['T1']
    T1 = T0.toarray()
    TargetX = data['TargetX']
    TargetY = data['TargetY']
    
    s_amp = [s['amplitude'] for s in Saccade]
    s_start = [s['start'] for s in Saccade]
    s_dur = [s['duration'] for s in Saccade]
    Time = T1.shape[1]

    # Find epoch time point by stimulus
    sti_epoch = []
    for x, y in zip(TargetX, TargetY):
        temp = np.abs(x) + np.abs(y)
        idx = np.argmax(temp > 1)
        sti_epoch.append(idx)
    
    # Find epoch time point by saccade    
    final_epoch = []
    for t_num in range(len(s_amp)):
        s_ampI = np.array(s_amp[t_num])
        temp_amp = np.zeros(np.size(s_ampI), dtype=bool)
        star_idx=s_start[t_num]<500; # before onset
        s_ampI[star_idx]=0;

        if np.max(s_ampI)>7:
            I = np.argmax(s_ampI)
            temp_amp[I] = True

        if sum(temp_amp)==0:
            c_label=0;
        else:
            c_label=1;

        if len(temp_amp) < 2:
            if temp_amp == True:
                temp_start = s_start[t_num]
                temp_dur = s_dur[t_num]
            elif temp_amp == False:
                temp_start = np.array([])
                temp_dur = np.array([])
        else:
            temp_start = s_start[t_num][temp_amp]
            temp_dur = s_dur[t_num][temp_amp]

        final_epoch.append([t_num, temp_amp, temp_start, c_label, temp_dur])
    
    final_epochtime = []
    for t_num in range(len(final_epoch)):

        temp_epoch = final_epoch[t_num]
        stif_epoch = sti_epoch[t_num]        
        if temp_epoch[3] == 0:
            continue

        temp_times = temp_epoch[2]
        temp_dur = temp_epoch[4]        

        final_ep=[]
        final_ep2=np.array([stif_epoch-eset[0], stif_epoch+eset[1]])
        if np.size(temp_times) < 2:
            temp_priod=[temp_times-eset[0], temp_times+eset[1], temp_epoch[0], temp_dur]
            final_ep.append(temp_priod)
        else:
            for len_epo in range(len(temp_times)):
                temp_priod=[temp_times[len_epo]-eset[0], temp_times[len_epo]+eset[1], temp_epoch[0], temp_dur]
                final_ep.append(temp_priod)

        final_epochtime.append(np.append(final_ep[0], final_ep2))

    final_epochtime=np.array(final_epochtime, dtype=object).astype(int)
    

    if reject_trials:
        valid = (final_epochtime[:,0] > 0) & (final_epochtime[:,1] <= Time)
        final_epochtime2 = final_epochtime[valid]
    else:
        final_epochtime2 = final_epochtime

    total_epoch.append([
        final_epoch,
        final_epochtime2,
        len(final_epochtime2),
        len(final_epochtime) - len(final_epochtime2),
        fname,
        sti_epoch
    ])
    
total_epoch=np.array(total_epoch,dtype=object)

C:\Users\Jisub\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:5386: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(arr)


In [3]:
# Save the result
os.chdir(r'C:\Users\Jisub\Desktop\Saccadic\python_code')
np.save('epoch_setting.npy',total_epoch)